<a href="https://colab.research.google.com/github/curiouscat7/Anomaly-detection/blob/main/Anomaly_detection_arla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv("arla_keyword_table.csv")

In [ ]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder



# Encode categorical columns ('ASIN' and 'Keyword Type') into numeric values
label_encoder = LabelEncoder()
df['PRODUCT_TITLE'] = label_encoder.fit_transform(df['PRODUCT_TITLE'])
df['Keyword Type'] = label_encoder.fit_transform(df['Keyword Type'])

# Encode the 'Keyword' column
df['Keyword'] = label_encoder.fit_transform(df['Keyword'])

# Select relevant columns for outlier detection
X = df[['PRODUCT_TITLE', 'Keyword Type', 'Keyword']]

# Initialize the Isolation Forest model
# You can adjust hyperparameters like 'n_estimators', 'contamination', etc.
clf = IsolationForest(contamination=0.05, random_state=42)

# Fit the model to your data and predict outliers
df['Outlier'] = clf.fit_predict(X)

# Filter the DataFrame to include only outliers (Outlier == -1)
outliers_df = df[df['Outlier'] == -1]

# Decode the 'Keyword' column to its original values if needed
outliers_df['Keyword'] = label_encoder.inverse_transform(outliers_df['Keyword'])

# Save the outliers to a CSV file
outliers_df.to_csv('outliers_arla.csv', index=False)

# Print or inspect the outliers if needed
print("Outliers:")
print(outliers_df)


/Users/samanwitamukherjee/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


Outliers:
       PRODUCT_TITLE  Keyword Type                           Keyword  Outlier
0                 13             2                      white cheese       -1
39                13             2             white cheese less fat       -1
71                13             1                       feta cheese       -1
76                13             1                     romano cheese       -1
77                13             1                  roquefort cheese       -1
...              ...           ...                               ...      ...
23279             80             1            yoplait protein yogurt       -1
23288             80             1  yocrunch fitness friendly yogurt       -1
23368             78             1              yoplait pouch yogurt       -1
23468             70             1              yoplait mango yogurt       -1
23477             70             1             yocrunch mango yogurt       -1

[1175 rows x 4 columns]


/var/folders/k6/2syb2mv54wn0mgvxrj8n8ltm0000gp/T/ipykernel_12561/1038981130.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outliers_df['Keyword'] = label_encoder.inverse_transform(outliers_df['Keyword'])


In [ ]:
pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.1 MB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade transformers

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/e1/9d/4d9fe5c3b820db10773392ac5f4a0c8dab668f70b245ce2ce09785166128/transformers-4.33.0-py3-none-any.whl.metadata
  Using cached transformers-4.33.0-py3-none-any.whl.metadata (119 kB)
  Obtaining dependency information for huggingface-hub<1.0,>=0.15.1 from https://files.pythonhosted.org/packages/7f/c4/adcbe9a696c135578cabcbdd7331332daad4d49b7c43688bc2d36b3a47d2/huggingface_hub-0.16.4-py3-none-any.whl.metadata
  Using cached huggingface_hub-0.16.4-py3-none-any.whl.metadata (12 kB)
  Using cached tokenizers-0.13.3-cp311-cp311-macosx_10_11_universal2.whl (4.0 MB)
  Using cached safetensors-0.3.3.tar.gz (35 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached transformers-4.33.0-py3-none-any.whl (7.6 MB)
Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
  Created wheel for safetensors: 

In [ ]:
pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 MB 3.0 MB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
 pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 6.1 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertModel
import torch

# Assuming you have loaded your DataFrame 'df' with your data

# Encode categorical columns (product_title and keyword_type)
label_encoders = {}
for column in ["PRODUCT_TITLE", "Keyword Type"]:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Define BERT tokenizer and model
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Encode the keywords using BERT embeddings
keyword_embeddings = []

max_length = 128  # Define your desired maximum length

for keyword in df['Keyword']:
    # Manually truncate the input sequence to the desired maximum length
    keyword = keyword[:max_length]
    inputs = tokenizer.encode_plus(keyword, add_special_tokens=True, padding=True, max_length=max_length, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
        hidden_states = outputs.last_hidden_state
    embeddings = hidden_states.mean(dim=1).squeeze().numpy()  # Mean pooling of BERT embeddings
    keyword_embeddings.append(embeddings)

X = np.hstack([df[["PRODUCT_TITLE", "Keyword Type"]].values, np.vstack(keyword_embeddings)])

# Train a suitable anomaly detection model (e.g., Isolation Forest, One-Class SVM)
from sklearn.ensemble import IsolationForest

clf = IsolationForest(contamination=0.05, random_state=42)  # Adjust the contamination parameter as needed
clf.fit(X)

# Predict anomalies (outliers)
df["anomaly_score"] = clf.decision_function(X)
df["is_anomaly"] = clf.predict(X)

# Save anomalies to a CSV file
anomalies = df[df["is_anomaly"] == -1]
anomalies.to_csv("anomalies.csv", index=False)

print("Anomalies saved to 'anomalies.csv'.")


Anomalies saved to 'anomalies.csv'.


In [ ]:
pip install --upgrade transformers


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertModel
import torch
from sklearn.svm import OneClassSVM  # Import the One-Class SVM module

# Assuming you have loaded your DataFrame 'df' with your data

# Encode categorical columns (product_title and keyword_type)
label_encoders = {}
for column in ["PRODUCT_TITLE", "Keyword Type"]:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Define BERT tokenizer and model
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Encode the keywords using BERT embeddings
keyword_embeddings = []

max_length = 128  # Define your desired maximum length

for keyword in df['Keyword']:
    # Manually truncate the input sequence to the desired maximum length
    keyword = keyword[:max_length]
    inputs = tokenizer.encode_plus(keyword, add_special_tokens=True, padding=True, max_length=max_length, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
        hidden_states = outputs.last_hidden_state
    embeddings = hidden_states.mean(dim=1).squeeze().numpy()  # Mean pooling of BERT embeddings
    keyword_embeddings.append(embeddings)

X = np.hstack([df[["PRODUCT_TITLE", "Keyword Type"]].values, np.vstack(keyword_embeddings)])

# Train a suitable anomaly detection model (e.g., One-Class SVM)
clf = OneClassSVM(nu=0.05)  # Adjust the nu parameter as needed
clf.fit(X)

# Predict anomalies (outliers)
df["anomaly_score"] = clf.decision_function(X)
df["is_anomaly"] = clf.predict(X)

# Save anomalies to a CSV file
anomalies = df[df["is_anomaly"] == -1]
anomalies.to_csv("anomalies_SVM.csv", index=False)

print("Anomalies saved to 'anomalies_SVM.csv'.")


Anomalies saved to 'anomalies_SVM.csv'.
